In [2]:
import os
import sys
sys.path.append("../../common/src")
sys.path.append("../src")
PROJECT_ID = os.getenv("PROJECT_ID")

In [3]:
from config.utils import set_agent_config
from common.models import (User, UserChat, QueryResult,
                           QueryEngine, UserPlan, PlanStep)
from common.models.llm import CHAT_AI
from common.models.agent import AgentCapability
from common.utils.logging_handler import Logger
from config import (get_model_config, PROVIDER_LANGCHAIN,
                    OPENAI_LLM_TYPE_GPT4,
                    VERTEX_LLM_TYPE_BISON_CHAT_LANGCHAIN,
                    OPENAI_LLM_TYPE_GPT4_LATEST)

from services.agents.routing_agent import run_intent, run_routing_agent
from services.agents.agents import BaseAgent
from langchain.agents import AgentExecutor
from services.agents.routing_agent import get_dispatch_prompt


INFO: [config/config.py:55 - <module>()] Namespace File not found, setting job namespace as default
INFO: [config/config.py:105 - <module>()] ENABLE_GOOGLE_LLM = True
INFO: [config/config.py:106 - <module>()] ENABLE_OPENAI_LLM = True
INFO: [config/config.py:107 - <module>()] ENABLE_COHERE_LLM = True
INFO: [config/config.py:108 - <module>()] ENABLE_GOOGLE_MODEL_GARDEN = True
INFO: [config/config.py:109 - <module>()] ENABLE_TRUSS_LLAMA2 = True
INFO: [config/vector_store_config.py:40 - <module>()] Default vector store = [matching_engine]
INFO: [config/vector_store_config.py:49 - <module>()] PG_HOST = [127.0.0.1]
INFO: [config/vector_store_config.py:50 - <module>()] PG_DBNAME = [pgvector]
ERROR: [config/vector_store_config.py:77 - <module>()] Cannot connect to pgvector instance at 127.0.0.1: (psycopg2.OperationalError) connection to server at "127.0.0.1", port 5432 failed: FATAL:  database "pgvector" does not exist

(Background on this error at: https://sqlalche.me/e/14/e3q8)
INFO: [utils/

In [4]:
AGENT_CONFIG = {
  "Agents":
  {
    "Router": {
      "llm_type": OPENAI_LLM_TYPE_GPT4,
      "agent_type": "langchain_Conversational",
      "agent_class": "RoutingAgent",
      "tools": ""
    },
    "Chat": {
      "llm_type": OPENAI_LLM_TYPE_GPT4,
      "agent_type": "langchain_Conversational",
      "agent_class": "ChatAgent",
      "tools": "search_tool,query_tool",
      "query_engines": "ALL"
    },
    "Task": {
      "llm_type": OPENAI_LLM_TYPE_GPT4_LATEST,
      "agent_type": "langchain_StructuredChatAgent",
      "agent_class": "TaskAgent",
      "tools": "ALL"
    },
    "Plan": {
      "llm_type": OPENAI_LLM_TYPE_GPT4_LATEST,
      "agent_type": "langchain_ZeroShot",
      "agent_class": "PlanAgent",
      "query_engines": "ALL",
      "tools": "ALL"
    }
  }
}

In [ ]:

def run_intent(agent_name):
  llm_service_agent = BaseAgent.get_llm_service_agent(agent_name)

  # load corresponding langchain agent and instantiate agent_executor
  langchain_agent = llm_service_agent.load_langchain_agent()
  intent_agent_tools = llm_service_agent.get_tools()
  print(f"Routing agent tools [{intent_agent_tools}]")

  agent_executor = AgentExecutor.from_agent_and_tools(
      agent=langchain_agent, tools=intent_agent_tools)

  # get dispatch prompt
  dispatch_prompt = get_dispatch_prompt(llm_service_agent)

  agent_inputs = {
    "input": dispatch_prompt + prompt,
    "chat_history": []
  }

  Logger.info("Running agent executor to get best matched route.... ")
  output, agent_logs = await agent_executor_arun_with_logs(
      agent_executor, agent_inputs)


In [ ]:
def parse_plan_step(text:str) -> dict:
  step_regex = re.compile(
      r"\d+\.\s.*\[(.*)\]\s?(.*)", re.DOTALL)
  matches = step_regex.findall(text)
  return matches

route, detail = parse_plan_step(routes[0])[0]
route

In [ ]:
prompt = "How do I apply for a driver's license?"
agent_inputs = {
    "input": prompt,
    "chat_history": []
  }
output = agent_executor.run(agent_inputs)
print(output)

In [ ]:
prompt = "What's the latest news in Jordan?"
agent_inputs = {
    "input": prompt,
    "chat_history": []
  }
output = agent_executor.run(agent_inputs)
print(output)

In [ ]:
prompt = "Compose and send an email to all the medicaid applicants that are missing income verification asking them to provide a pay stub from their employers"
agent_inputs = {
    "input": prompt,
    "chat_history": []
  }
output = agent_executor.run(agent_inputs)
print(output)